## Логарифмическая регрессия

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/home/mrl/Desktop/ML_home_work/1/adult.csv')

In [4]:
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
age                48842 non-null int64
workclass          48842 non-null object
fnlwgt             48842 non-null int64
education          48842 non-null object
educational-num    48842 non-null int64
marital-status     48842 non-null object
occupation         48842 non-null object
relationship       48842 non-null object
race               48842 non-null object
gender             48842 non-null object
capital-gain       48842 non-null int64
capital-loss       48842 non-null int64
hours-per-week     48842 non-null int64
native-country     48842 non-null object
income             48842 non-null object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [6]:
data.isnull().all()

age                False
workclass          False
fnlwgt             False
education          False
educational-num    False
marital-status     False
occupation         False
relationship       False
race               False
gender             False
capital-gain       False
capital-loss       False
hours-per-week     False
native-country     False
income             False
dtype: bool

In [7]:
from sklearn.linear_model import LogisticRegression

In [8]:
# строим модель
regr = LogisticRegression()

In [9]:
# попробуем выбрать какие-нибудь признаки
selectedColumns = data[ [ 'age', 'education', 'educational-num', 'gender',  'hours-per-week', 'income' ] ]
selectedColumns.head()

,age,education,educational-num,gender,hours-per-week,income
0,25,11th,7,Male,40,<=50K
1,38,HS-grad,9,Male,50,<=50K
2,28,Assoc-acdm,12,Male,40,>50K
3,44,Some-college,10,Male,40,>50K
4,18,Some-college,10,Female,30,<=50K


In [10]:
data.income.value_counts()

<=50K    37155
>50K     11687
Name: income, dtype: int64

In [11]:
X = pd.get_dummies( selectedColumns, columns = [ 'gender', 'education' ] )

# столбец income является целевой переменной y, удаляем его из X
del X['income']

In [12]:
X.head()

,age,educational-num,hours-per-week,gender_Female,gender_Male,education_10th,education_11th,education_12th,education_1st-4th,education_5th-6th,...,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Preschool,education_Prof-school,education_Some-college
0,25,7,40,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,38,9,50,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,28,12,40,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,44,10,40,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,18,10,30,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [13]:
# переведем значения столбца income в числа, оставив один столбец

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit( data['income'] )
le.classes_


array(['<=50K', '>50K'], dtype=object)

In [14]:
le.transform(data['income'])

array([0, 0, 1, ..., 0, 0, 1])

In [15]:
le.transform(['<=50K', '>50K'])

array([0, 1])

In [16]:
# записываем в переменную y преобразованный столбец income

y = pd.Series( data = le.transform( data['income'] ) )
y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int64

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [24]:
# обучаем модель

regr.fit(X_train, y_train)


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:760: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [26]:
y_predict = regr.predict(X_test)

In [55]:
y_predict_proba = regr.predict_proba(X_test)

In [50]:
#предсказнные зарплаты 
y_predict

array([0, 0, 1, ..., 0, 0, 0])

In [60]:
# вероятность события 1
y_predict_proba[:,1]

array([0.01366823, 0.20720154, 0.81056427, ..., 0.05989352, 0.17892717,
       0.41271099])

In [68]:
#коэффициент детерминации

score = regr.score(X_test, y_test)
print("Точноть предсказания: %.2f" % (score * 100) + ' %' )

Точноть предсказания: 80.37 %


## SVM

In [69]:
from sklearn.svm import SVC

In [70]:
svm = SVC()

In [71]:
svm.fit(X_train, y_train)

SVC()

In [72]:
score = svm.score(X_test, y_test)
print("Точноть предсказания: %.2f" % (score * 100) + ' %' )

Точноть предсказания: 80.33 %
